---

Title: Data Storage, Integration and Enrichment - With PostgreSQL

---

---
### 🛠️ Importing Libraries

In [1]:
# Basic/Standard libraries to manage data
import numpy as np
import pandas as pd

# To use PostgreSQL in python environment
# pip install psycopg2 # (only if you haven't installed this library in your pc)
# pip install sqlalchemy # (only if you haven't installed this library in your pc)
import psycopg2 # A PostgreSQL adapter for Python, for interacting with PostgreSQL databases
from psycopg2 import sql # Importing the sql module from psycopg2 to help construct SQL queries safely and dynamically
from sqlalchemy import create_engine # Importing the create_engine function from SQLAlchemy, which helps establish database connections using various database engines

# pip install tqdm # (only if you haven't installed this library in your pc)
from tqdm import tqdm # To add a progress bar when you use a for cycle (useful when you want to take trace of the iterations in a cycle or its execution time)

---
### 🗃️ Data Storage & Integration
In our case, the tables Hackers, Targets, and Attacks have been designed to represent attackers, victims, and the relationships between them, respectively. With the rigid structure of relational databases, the use of primary and foreign keys, and PostgreSQL as the database management system, integrity, scalability, and ease of data analysis are ensured. This approach is particularly advantageous for scenarios where the data schema is fixed and involves processing large datasets.

#### **Advantages of storing data in a relational database**
- ##### **Why a relational database is ideal?**
   1. **Structure and integrity:**
      - Fixed schema ensures consistency.
      - Primary and foreign keys enforce referential integrity, preventing duplicates or invalid data.

   2. **Query efficiency:**
      - SQL enables complex queries for aggregation and filtering, essential for EDA.

   3. **Processing efficiency:**
      - Relational databases handle large datasets efficiently.

   4. **Scalability and management:**
      - Easily manages 40,000 observations per table without performance issues.

- ##### **Why PostgreSQL is an ideal choice?**
   1. **Performance and scalability:**
      - Optimized for large datasets.

   2. **Support for complex operations:**
      - Offers advanced SQL functionalities for analytical tasks.

   3. **Concurrency management:**
      - MVCC enables simultaneous data access.

   4. **Open-source and extensibility:**
      - Free and customizable with extensions.
      
   5. **Relationship management:**
      - Full support for foreign keys, making it suitable for complex relational models.
      
#### **Database Engineering**
- ##### **Database Design Considerations**
   To ensure clarity regarding the design of our database, we emphasize that it has been tailored specifically to the nature of the data we collected. Here are the key considerations that guided our design decisions:

   **Static Nature of the Data**
   - The data we are working with is **static** and will not be updated in the future.
   - The primary goal of this project is to answer the research question defined at the beginning, relying on **Exploratory Data Analysis (EDA)** of the current dataset.

   **Relationships Between Entities**
   Based on the structure and characteristics of our data:
   - **One hacker** is related to **one and only one attack**, which in turn is related to **one and only one target**.
   - Conversely, **one target** is related to **one and only one attack**, which is subsequently related to **one and only one hacker**.

   **Implications on Database Design**
   - This strict **one-to-one relationship** between hackers, attacks, and targets simplifies the design of our database, as each entity corresponds uniquely to another.
   - **Foreign key relationships** remain critical to enforce referential integrity and maintain the linkage between the entities.

   This clear understanding of the data and its relationships ensures that our database is both optimized for the specific analysis at hand and appropriately structured to reflect the unique nature of the dataset.

- ##### **Relation between entities**
   - **Entity `Hackers`:** Represents attackers initiating an attack. Here there are information about hackers. This info are collected (or more pecisely generated) tanks to the use of an API (for more info see documentation/report)
      - **Primary key:** `att_ip`

   - **Entity `Targets`:** Represents the attack victims. Here there are information about victims. Also this info are collected (or more pecisely generated) tanks to the use of the same API mentioned before.
      - **Primary key:** `vic_ip`
  
   - **Relation `Attacks`:** This table connects `Hackers` to `Targets` by representing specific attacks. It includes:
      - **Foreign key 1:** `Source IP Address` (linked to `Hackers.att_ip`)
      - **Foreign key 2:** `Destination IP Address` (linked to `Targets.vic_ip`)

- ##### **Entity-Relationship (ER) Model**
   - **Entities:**
      - `Hackers(att_ip, ... other attributes)`
      - `Targets(vic_ip, ... other attributes)`

   - **Relation:**
      - `Attacks(Source IP Address, Destination IP Address, ... other attributes)`

   - **Cardinality:**
      - Each **Hacker** is related to **one and only one Attack** (1:1).
      - Each **Attack** is related to **one and only one Target** (1:1).  

- ##### **Logical model**
   - **Tables:**
      1. `Hackers(att_ip [PK], att_continent_name, att_country_name, ...)`
      2. `Targets(vic_ip [PK], vic_continent_name, vic_country_name, ...)`
      3. `Attacks(Source IP Address [FK -> Hackers.att_ip], Destination IP Address [FK -> Targets.vic_ip], Protocol, Severity Level, Attack Type, ...)`
      
   - **Relationships:**
      - `Hackers.att_ip` → `Attacks.Source IP Address`
      - `Targets.vic_ip` → `Attacks.Destination IP Address`

#### Database Configuration

In [2]:
# Database credentials
db_config = {
    "dbname": "DataMan_DB",       # Name of your database
    "user": "postgres",           # PostgreSQL user
    "password": "dataAcc0",       # User password
    "host": "localhost",          # Database host (local)
    "port": "5432"                # Default port for PostgreSQL
}

#### Function to import CSV files into PostgreSQL database
The function `import_CSV_as_TABLE` imports a CSV file into a PostgreSQL database as a table using the `SQLAlchemy` library for database interaction and `Pandas` for reading and handling the CSV data. 

##### **Key Features**:
1. **Database Connection**:
   - Establishes a connection to the PostgreSQL database using a SQLAlchemy engine created from the `db_config` dictionary, which contains database credentials (`user`, `password`, `host`, `port`, and `dbname`).

2. **CSV to DataFrame**:
   - Reads the specified CSV file (`csv_file_path`) into a Pandas DataFrame for easy manipulation.

3. **Table Creation**:
   - Writes the DataFrame into the database as a new table (`table_name`), replacing any existing table with the same name (`if_exists='replace'`).

4. **Error Handling**:
   - Catches and reports any errors that occur during the table creation or data import process.

5. **Resource Management**:
   - Closes the database engine after the operation is completed to free resources.

In [3]:
def import_CSV_as_TABLE(csv_file_path, table_name, db_config):

    # SQLAlchemy engine creation
    engine = create_engine(
        f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
    )

    # From CSV to a Pandas dataframe
    df = pd.read_csv(csv_file_path)

    # Write the dataframe on the PostgreSQL database (DataMan_DB) as table
    try:
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Table '{table_name}' created and populated successfully!")
    except Exception as e:
        print("Error while importing:", e)
    
    # Close the engine
    engine.dispose()

#### Building process of our database

In [ ]:
# CSV files path
#csv_files_path = r"D:\UniVault\UniProjects\DataScience\Year_1\DataMan\data\csv_file"

# A dictionarie that contain as keys the names of the files.csv and as values the names of the new tables that will build in the database
#diz_fileNames_tableNames = {"updated_cybersecurity_attacks.csv":"Attacks",
#                            "attakers_csv_file.csv":"Hackers",
#                            "victims_csv_file.csv":"Targets"
#                            }

# Importing files into the database
#for fileName in tqdm(diz_fileNames_tableNames, desc = "Import files"):
#    tableName = diz_fileNames_tableNames[fileName]
#    csv_file_path = f"{csv_files_path}\\{fileName}"
#    import_CSV_as_TABLE(csv_file_path, tableName, db_config)

Import files:  33%|███▎      | 1/3 [00:03<00:06,  3.15s/it]

Table 'Attacks' created and populated successfully!


Import files:  67%|██████▋   | 2/3 [00:06<00:03,  3.14s/it]

Table 'Hackers' created and populated successfully!


Import files: 100%|██████████| 3/3 [00:08<00:00,  2.81s/it]

Table 'Targets' created and populated successfully!


---
### 🧩 Data Enrichment and Quering

#### Function to connect and send queries
The `execute_query` function is a utility for executing SQL queries on a PostgreSQL database using the `psycopg2` library. 

##### **Key Features**:

1. **Database Connection**:
   - Establishes a connection to the PostgreSQL database using `db_config`, which contain the database credentials.

2. **Query Execution**:
   - Executes the provided SQL query (`query`) using a cursor object.

3. **Result Handling**:
   - If the query is a `SELECT` statement, it retrieves and prints all results row by row.
   - For other query types (e.g., `INSERT`, `UPDATE`, `DELETE`), it commits the changes to the database.

4. **Error Handling**:
   - Catches and displays any exceptions that occur during the query execution process.

5. **Resource Management**:
   - Utilizes Python's `with` statement to ensure that the database connection and cursor are properly closed after execution, even in case of errors.

##### **Use Case**:
This function is ideal if you just need to see the output of a given query.

In [4]:
def execute_query(query):
    try:
        # Conneciton to the database
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cur:
                cur.execute(query)  # Executes the query
                # If it is a SELECT, retrieve the results
                if query.strip().lower().startswith("select"):
                    results = cur.fetchall()
                    for row in results:
                        print(row)
                else:
                    conn.commit()  # Save changes for queries like INSERT or UPDATE
                    print("Query successful!")
    except Exception as e:
        print("Error executing query:", e)

#### Query examples

In [5]:
# See all of the tables in your database
if __name__ == "__main__":
    # Query
    ExtractTables = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
    execute_query(ExtractTables)

('Attacks',)
('Hackers',)
('Targets',)


In [6]:
# Selecting the first 10 attacks from the table Attacks
if __name__ == "__main__":
    # Query
    First10Attacks = 'SELECT * FROM "Attacks" LIMIT 10;'
    execute_query(First10Attacks)

(0, '2023-05-30 06:33:58', '103.216.15.12', '84.9.164.252', 31225, 17616, 'ICMP', 503, 'Data', 'HTTP', 'Qui natus odio asperiores nam. Optio nobis iusto accusamus ad perferendis esse at. Asperiores neque at ad.\nMaiores possimus ipsum saepe vitae. Ad possimus veritatis.', 'IoC Detected', 28.67, 'no', 'Malware', 'Known Pattern B', 'Logged', 'Low', 'Reyansh Dugal', 'Segment A', 'Jamshedpur, Sikkim', '150.9.97.135', 'Log Data', 'No Data', 'Server', 'Mozilla', 'Windows', 2023, 5, 30, 6, 33, 58, 1)
(1, '2020-08-26 07:08:30', '78.199.217.198', '66.191.137.154', 17245, 48166, 'ICMP', 1174, 'Data', 'HTTP', 'Aperiam quos modi officiis veritatis rem. Omnis nulla dolore perspiciatis.\nIllo animi mollitia vero voluptates error ad. Quidem maxime eaque optio a. Consectetur quasi veniam et totam culpa ullam.', 'IoC Detected', 51.5, 'no', 'Malware', 'Known Pattern A', 'Blocked', 'Low', 'Sumer Rana', 'Segment B', 'Bilaspur, Nagaland', 'No proxy', 'Log Data', 'No Data', 'Firewall', 'Mozilla', 'Windows',

In [7]:
# Extracts the name of columns from table Attacks
if __name__ == "__main__":
    # Query
    ColName = "SELECT column_name FROM information_schema.columns WHERE table_name = 'Attacks';" 
    execute_query(ColName)

('Unnamed: 0',)
('Source Port',)
('Destination Port',)
('Packet Length',)
('Anomaly Scores',)
('Year',)
('Month',)
('Day',)
('Hour',)
('Minute',)
('Second',)
('DayOfWeek',)
('Device/OS',)
('Alerts/Warnings',)
('Attack Type',)
('Attack Signature',)
('Action Taken',)
('Severity Level',)
('User Information',)
('Network Segment',)
('Geo-location Data',)
('Proxy Information',)
('Firewall Logs',)
('Timestamp',)
('Source IP Address',)
('Destination IP Address',)
('IDS/IPS Alerts',)
('Log Source',)
('Protocol',)
('Browser',)
('Packet Type',)
('Traffic Type',)
('Payload Data',)
('Malware Indicators',)


#### Function to executes a SQL query on a PostgreSQL database and to returns the results as a Pandas DataFrame
The `execute_query_to_dataframe` function executes a SQL query on a PostgreSQL database and returns the results as a Pandas DataFrame. 

##### **Key Features**:

1. **Database Connection**:
   - Connects to the PostgreSQL database using the provided `db_config` dictionary, which contains credentials.

2. **Query Execution**:
   - Executes the given SQL query (`query`) and fetches the results using `pd.read_sql_query`, which converts the query results directly into a Pandas DataFrame.

3. **Error Handling**:
   - Catches and displays any errors that occur during execution, returning `None` in case of failure.

4. **Resource Management**:
   - Ensures the database connection is properly closed after the query execution using Python's `with` statement.

##### **Use Case**:
This function is ideal for scenarios where query results need to be further analyzed, visualized, or manipulated using Pandas, providing a seamless bridge between SQL and Python's data analysis ecosystem.

In [8]:
def execute_query_to_dataframe(query, db_config):
    try:
        # Conneciton to the database
        with psycopg2.connect(**db_config) as conn:
            df = pd.read_sql_query(query, conn) # Execute the query and extract it in a pandas dataframe
        return df
    except Exception as e:
        print("Error executing query:", e)
        return None

#### Extracting and saving data for the EDA

The query `extractData4EDA` is designed to extract and enrich data from three tables: **Attacks** (the dataset to be enriched), **Targets**, and **Hackers** (the tables providing enrichment data). 

This query plays a critical role in the **data preparation step**, which involves processing and cleaning the raw data to ensure it is ready for **Exploratory Data Analysis (EDA)**. By combining these tables, the enriched dataset provides a more comprehensive view, enabling deeper and more meaningful insights during the analysis phase.

##### **Purpose of the Query**:
1. **Data Enrichment**:
   - Combines relevant fields from the three tables using **LEFT JOINs**, ensuring all records from the `Attacks` table are retained, even if there are no matching records in the `Targets` or `Hackers` tables. This approach ensures we preserve the integrity of the raw data for complete analysis.

2. **Feature Selection**:
   - Selects specific columns from the `Targets`, `Hackers`, and `Attacks` tables that are relevant for analysis. This includes information such as geolocation, threat scores, and attack attributes.

3. **Preparation for Cleaning and Analysis**:
   - The resulting dataset serves as a single, consolidated source of truth for subsequent data cleaning, transformation, and exploratory tasks.

##### **Why Process and Clean Data Before EDA?**
- **Ensure Data Quality**: Cleaning the data (e.g., handling missing values, removing duplicates, or correcting inconsistencies) improves its reliability.
- **Streamline EDA**: By resolving data issues upfront, we can focus on uncovering meaningful patterns and insights without being hindered by errors or noise in the data.
- **Optimize Analysis**: Consolidated and cleaned data enables more accurate statistical computations and visualizations, enhancing the overall EDA process.

This query helps establish a robust foundation for extracting knowledge from the data, aligning with best practices in data science workflows.

In [9]:
extractData4EDA = """
SELECT
    -- Columns from 'Targets'
    t.vic_ip, t.vic_continent_code, t.vic_continent_name, t.vic_country_code2,
    t.vic_country_code3, t.vic_country_name, t.vic_country_name_official,
    t.vic_is_eu, t.vic_state_prov, t.vic_state_code, t.vic_district, t.vic_city,
    t.vic_zipcode, t.vic_latitude, t.vic_longitude, t.vic_threat_score, t.vic_is_tor,
    t.vic_is_proxy, t.vic_proxy_type, t.vic_is_anonymous, t.vic_is_known_attacker,
    t.vic_is_spam, t.vic_is_bot, t.vic_is_cloud_provider, t.vic_message,
    -- Columns from 'Hackers'
    h.att_ip, h.att_continent_code, h.att_continent_name, h.att_country_code2,
    h.att_country_code3, h.att_country_name, h.att_country_name_official,
    h.att_is_eu, h.att_state_prov, h.att_state_code, h.att_district, h.att_city,
    h.att_zipcode, h.att_latitude, h.att_longitude, h.att_threat_score, h.att_is_tor,
    h.att_is_proxy, h.att_proxy_type, h.att_is_anonymous, h.att_is_known_attacker,
    h.att_is_spam, h.att_is_bot, h.att_is_cloud_provider, h.att_message,
    -- Columns from 'Attacks'
    a."Source IP Address", a."Destination IP Address", a."Protocol", a."Packet Type",
    a."Packet Length", a."Traffic Type", a."Malware Indicators", a."Anomaly Scores", a."Alerts/Warnings",
    a."Attack Type", a."Action Taken", a."Severity Level", a."Log Source", a."Browser", a."Device/OS",
    a."Year", a."Month", a."Day"
FROM 
    "Attacks" a
LEFT JOIN 
    "Targets" t ON a."Destination IP Address" = t.vic_ip
LEFT JOIN 
    "Hackers" h ON a."Source IP Address" = h.att_ip;
"""

if __name__ == "__main__":
    data4EDA = execute_query_to_dataframe(extractData4EDA, db_config)
    if data4EDA is not None:
        print(data4EDA.head())

C:\Users\loren\AppData\Local\Temp\ipykernel_13324\2042812010.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn) # Execute the query and extract it in a pandas dataframe


            vic_ip vic_continent_code vic_continent_name vic_country_code2  \
0     84.9.164.252                 EU             Europe                GB   
1   66.191.137.154               None      North America                US   
2    198.219.82.17               None      North America                US   
3  101.228.192.255                 AS               Asia                CN   
4  189.243.174.238               None      North America                MX   

  vic_country_code3 vic_country_name  \
0               GBR   United Kingdom   
1               USA    United States   
2               USA    United States   
3               CHN            China   
4               MEX           Mexico   

                           vic_country_name_official vic_is_eu  \
0  United Kingdom of Great Britain and Northern I...     False   
1                           United States of America     False   
2                           United States of America     False   
3                         

##### Useful checks on the final dataset

In [ ]:
# Shape of the dataset 
data4EDA.shape

(40000, 68)

In [11]:
# Takes the columns
data4EDA.columns

Index(['vic_ip', 'vic_continent_code', 'vic_continent_name',
       'vic_country_code2', 'vic_country_code3', 'vic_country_name',
       'vic_country_name_official', 'vic_is_eu', 'vic_state_prov',
       'vic_state_code', 'vic_district', 'vic_city', 'vic_zipcode',
       'vic_latitude', 'vic_longitude', 'vic_threat_score', 'vic_is_tor',
       'vic_is_proxy', 'vic_proxy_type', 'vic_is_anonymous',
       'vic_is_known_attacker', 'vic_is_spam', 'vic_is_bot',
       'vic_is_cloud_provider', 'vic_message', 'att_ip', 'att_continent_code',
       'att_continent_name', 'att_country_code2', 'att_country_code3',
       'att_country_name', 'att_country_name_official', 'att_is_eu',
       'att_state_prov', 'att_state_code', 'att_district', 'att_city',
       'att_zipcode', 'att_latitude', 'att_longitude', 'att_threat_score',
       'att_is_tor', 'att_is_proxy', 'att_proxy_type', 'att_is_anonymous',
       'att_is_known_attacker', 'att_is_spam', 'att_is_bot',
       'att_is_cloud_provider', 'att_m

##### Saving of the final dataframe

In [ ]:
full_destination_path = r""
data4EDA.to_csv(full_destination_path, index = False)
print("File successfully saved!!!")
print(f"{"Your file location:"}{full_destination_path}")

File successfully saved!!!
Your file location:D:\UniVault\UniProjects\DataScience\Year_1\DataMan\data\csv_file\final_df_ExtractedWithPostgreSQL.csv


---
### ⏭️ Next Step: 
$\rightarrow$ Data Preparation (see 3_Data_Prepration.ipynb)